In [ ]:
import os
import re
import json
import time
import math
import requests
import pandas as pd
from glob import glob
from bs4 import BeautifulSoup
from rapidfuzz import process, fuzz
from util.logger import setup_logger

# ===== 로거 설정 =====
logger = setup_logger(__name__)

# ----------------------------------------------------
# 🔹 경로 설정
# ----------------------------------------------------
BASE_DIR = "C:/ESG_Project1/file/"
KMA_DIR = os.path.join(BASE_DIR, "KMA_data_file/")
SOLAR_DIR = os.path.join(BASE_DIR, "solar_data_file/")
OUT_CSV = os.path.join(SOLAR_DIR, "train_data.csv")

CACHE_JSON = os.path.join(BASE_DIR, "json/mapping_cache.json")
REGION_FIX_JSON = os.path.join(BASE_DIR, "json/region_fix.json")

# ----------------------------------------------------
# 🔹 공통 유틸
# ----------------------------------------------------
def sniff_delimiter(path):
    with open(path, "rb") as f:
        raw = f.read(2048)
    text = raw.decode("utf-8", errors="ignore")
    return "," if text.count(",") >= text.count("\t") else "\t"


def read_csv_safe(path):
    delim = sniff_delimiter(path)
    try:
        return pd.read_csv(path, encoding="utf-8", delimiter=delim)
    except UnicodeDecodeError:
        logger.warning(f"{path} UTF-8 실패 → cp949로 재시도")
        return pd.read_csv(path, encoding="cp949", delimiter=delim)


# ----------------------------------------------------
# 🔹 남동발전 발전소 지역 매핑 크롤러
# ----------------------------------------------------
def crawl_mapping():
    URL = "https://www.koenergy.kr/kosep/hw/fr/ov/ovhw25/main.do?menuCd=FN060202"
    logger.info(f"크롤링 시작: {URL}")

    headers = {"User-Agent": "Mozilla/5.0"}
    res = requests.get(URL, headers=headers, timeout=10)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "html.parser")

    tables = soup.find_all("table", class_="table_list2")
    if len(tables) < 2:
        logger.error("❌ class='table_list2' 테이블이 충분하지 않습니다.")
        raise RuntimeError("테이블 부족")

    table = tables[1]
    mapping = {}
    for tr in table.find_all("tr"):
        tds = [td.get_text(strip=True) for td in tr.find_all("td")]
        if len(tds) < 2:
            continue
        name, region = tds[0], tds[1]
        if "태양광" not in name:
            continue
        name = name.replace("발전소", "").replace(" ", "").strip()
        mapping[name] = region

    os.makedirs(os.path.dirname(CACHE_JSON), exist_ok=True)
    with open(CACHE_JSON, "w", encoding="utf-8") as f:
        json.dump(mapping, f, ensure_ascii=False, indent=2)

    logger.info(f"{len(mapping)}개 항목 크롤링 완료 → {CACHE_JSON}")
    return mapping


# ----------------------------------------------------
# 🔹 데이터 처리 함수
# ----------------------------------------------------
def normalize_columns(df):
    df.columns = df.columns.str.strip()
    if "발전구분" not in df.columns:
        expected = ["발전구분", "호기", "일자"] + [f"{i}시 발전량(MWh)" for i in range(1, 25)]
        df = df.iloc[:, :len(expected)]
        df.columns = expected
    df["발전구분"] = df["발전구분"].astype(str).str.strip()
    df["일자"] = pd.to_datetime(df["일자"], errors="coerce")
    return df


def get_hour_cols(df):
    return [c for c in df.columns if re.match(r"^\s*\d{1,2}시", c)]


# ----------------------------------------------------
# 🔹 전체 처리 파이프라인 (배열 기반)
# ----------------------------------------------------
def process_all_data():
    all_data = {}  # 메모리 내 저장소

    # 1️⃣ 발전량 데이터 수집
    solar_files = glob(os.path.join(SOLAR_DIR, "*.csv")) + glob(os.path.join(SOLAR_DIR, "*.CSV"))
    if not solar_files:
        logger.error("CSV 파일이 없습니다.")
        raise FileNotFoundError("발전량 CSV 없음")

    solar_frames = []
    for f in solar_files:
        try:
            tmp = read_csv_safe(f)
            tmp = normalize_columns(tmp)
            tmp["파일출처"] = os.path.basename(f)
            solar_frames.append(tmp)
            logger.info(f"불러옴: {os.path.basename(f)} ({len(tmp)}행)")
        except Exception as e:
            logger.warning(f"{os.path.basename(f)} 실패: {e}")

    df_solar = pd.concat(solar_frames, ignore_index=True)
    df_solar = df_solar.drop_duplicates()

    # melt 처리
    hour_cols = get_hour_cols(df_solar)
    df_solar_long = df_solar.melt(
        id_vars=["발전구분", "호기", "일자"],
        value_vars=hour_cols,
        var_name="시간대",
        value_name="발전량(MWh)"
    )
    df_solar_long["시간"] = df_solar_long["시간대"].str.extract(r"(\d{1,2})").astype(int)
    df_solar_long["일시"] = df_solar_long["일자"] + pd.to_timedelta(df_solar_long["시간"] - 1, "h")
    all_data["solar"] = df_solar_long

    # 2️⃣ 지역 매핑 로드
    if os.path.exists(CACHE_JSON):
        with open(CACHE_JSON, "r", encoding="utf-8") as f:
            mapping = json.load(f)
        logger.info(f"기존 캐시 사용 ({len(mapping)}건)")
    else:
        mapping = crawl_mapping()
    all_data["mapping"] = mapping

    # 3️⃣ 기상 데이터 수집
    weather_files = sorted(glob(os.path.join(KMA_DIR, "OBS_ASOS_TIM_20*.csv")))
    weather_frames = []
    for wf in weather_files:
        try:
            tmp = read_csv_safe(wf)
            tmp["일시"] = pd.to_datetime(tmp["일시"], errors="coerce")
            tmp = tmp[["지점", "일시", "기온(°C)", "강수량(mm)", "일조(hr)", "일사(MJ/m2)"]]
            weather_frames.append(tmp)
            logger.info(f"기상 불러옴: {os.path.basename(wf)} ({len(tmp)}행)")
        except Exception as e:
            logger.error(f"{os.path.basename(wf)} 실패: {e}")
    df_weather = pd.concat(weather_frames, ignore_index=True)
    all_data["weather"] = df_weather

    # 4️⃣ 발전량 + 기상데이터 병합 (메모리 내)
    merged = pd.merge(
        all_data["solar"],
        all_data["weather"],
        on="일시",
        how="left"
    )
    weather_cols = ["기온(°C)", "강수량(mm)", "일조(hr)", "일사(MJ/m2)"]
    merged[weather_cols] = merged[weather_cols].fillna(0)
    all_data["merged_final"] = merged

    return all_data


# ----------------------------------------------------
# 🔹 실행부
# ----------------------------------------------------
if __name__ == "__main__":
    logger.info("=== 데이터 통합 시작 ===")
    all_data = process_all_data()

    # 마지막에만 CSV 저장
    all_data["merged_final"].to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
    logger.info(f"✅ 최종 병합 완료 → {OUT_CSV}")


ModuleNotFoundError: No module named 'bs4'